# Evaluate Kinyarwanda-trained model on FLEURS – Wolof

In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch, librosa, jiwer

# Load Wolof test set
fleurs = load_dataset("google/fleurs", "wolof")
test_set = fleurs["test"].select(range(50))

# Load Kinyarwanda-trained model
model = Wav2Vec2ForCTC.from_pretrained("./models/wav2vec2-base")
processor = Wav2Vec2Processor.from_pretrained("./models/wav2vec2-base")
model.eval()

refs, preds = [], []
for sample in test_set:
    speech, _ = librosa.load(sample["audio"]["path"], sr=16000)
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    ids = torch.argmax(logits, dim=-1)
    pred = processor.batch_decode(ids)[0]
    preds.append(pred)
    refs.append(sample["raw_transcription"])

print("WER:", jiwer.wer(refs, preds))
print("CER:", jiwer.cer(refs, preds))